## Challenge: If a tree falls in the forest...

Comparing Decision Tree and Random Forest algorithms by runtime.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn import ensemble
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [2]:
child_care = pd.read_csv(r'C:\Users\Omistaja\nys-children-in-foster-care-annually\children-in-foster-care-annually-beginning-1994.csv').dropna()

The purpose of this data set is to provide information on the total number of admissions, discharges, and children in foster care, the type of care, and total Child Protective Services (CPS) reports indicated during period. the data set can be downloaded here https://www.kaggle.com/new-york-state/nys-children-in-foster-care-annually 

For this excercise we will tryt to predict Discharges using regressor. 

In [3]:
child_care.head()

,County,Year,Adoptive Home,Agency Operated Boarding Home,Approved Relative Home,Foster Boarding Home,Group Home,Group Residence,Institution,Supervised Independent Living,Other,Total Days In Care,Admissions,Discharges,Children In Care,Number of Children Served,Indicated CPS Reports
0,ALBANY,2017,0,965,2598,48637,5207,1488,16017,692,232,75836,158,130.0,199,402,602.0
1,ALLEGANY,2017,0,585,5596,11320,0,0,2615,285,0,20401,22,46.0,58,77,89.0
2,BROOME,2017,0,393,6171,53256,8419,451,13870,3828,468,86856,136,108.0,241,385,1016.0
3,CATTARAUGUS,2017,0,1006,2747,14925,132,0,3949,106,256,23121,45,80.0,66,113,320.0
4,CAYUGA,2017,0,337,0,15543,92,155,5067,455,0,21649,40,42.0,58,120,208.0


In [4]:
# lets avoid the only non numeric column
child_care = child_care.drop('County', 1)

In [17]:
child_care.shape

(1403, 16)

In [5]:
# Check data type
child_care.dtypes

Year                                int64
Adoptive Home                       int64
Agency Operated Boarding Home       int64
 Approved Relative Home             int64
 Foster Boarding Home               int64
 Group Home                         int64
 Group Residence                    int64
Institution                         int64
 Supervised Independent Living      int64
Other                               int64
Total Days In Care                  int64
Admissions                          int64
Discharges                        float64
Children In Care                    int64
Number of Children Served           int64
Indicated CPS Reports             float64
dtype: object

In [6]:
# Let's use all variables and apply pca 
X = child_care.drop('Discharges', 1)
Y = child_care['Discharges']

In [7]:
# pca requires this 
X_nor = StandardScaler().fit_transform(X)

In [8]:
# five components explain allmost all of the varience 
pca =PCA(n_components=5)

In [9]:
X_reg = pca.fit_transform(X_nor)
print(pca.explained_variance_ratio_)

[0.77383099 0.07464166 0.06257534 0.04459756 0.02697831]


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_reg, Y, test_size = 0.2, random_state = 0)

### First using decision tree regressor

In [11]:
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(x_train, y_train)
y_ = regressor.predict(x_test)

import time
start_time = time.clock()
print("--- %s seconds ---" % (time.clock() - start_time))

--- 3.413330420624708e-05 seconds ---


In [12]:
score_tree = cross_val_score(regressor, X_reg, Y, cv=10)
print(score_tree)

[0.98415518 0.99257293 0.98497597 0.99485666 0.97473199 0.98455885
 0.97402008 0.99235562 0.99558184 0.99329543]


### Using random forest regressor 

In [13]:
x_train, y_train = make_regression(n_features=5, n_informative=2, random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=5, random_state=0)
regr.fit(x_train, y_train)
Y_ = regr.predict(x_test)
import time
start_time = time.clock()
print("--- %s seconds ---" % (time.clock() - start_time))
#round(time.time() - start_time, 2)

--- 3.925329983719972e-05 seconds ---


In [14]:
score_forest = cross_val_score(regr, X_reg, Y, cv=10)
print(score_forest)

[0.96694004 0.99648392 0.9682422  0.9890833  0.99339856 0.9898025
 0.97701399 0.98006366 0.99864959 0.99498602]


Both models have very igh accuracy and the Random forest algorithm takes slightly more time to excute. This data sets has only 1403 rows and I expect that the difference in the time it takes the two algorithms to excute will be much bigger with larger data sets.   